In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
penguins = pd.read_csv('C:/Users/HP/Desktop/Classification_Regression_Methods/noisy_datasets/noisy_datasets/penguins.csv')

dataset1 = pd.read_csv('C:/Users/HP/Desktop/Classification_Regression_Methods/noisy_datasets/noisy_datasets/emissions_by_country.csv')
dataset2 = pd.read_csv('C:/Users/HP/Desktop/Classification_Regression_Methods/noisy_datasets/noisy_datasets/diamond.csv')



In [10]:
print(penguins.describe())

       calorie requirement  average sleep duration  bill_length_mm  \
count           344.000000              344.000000      337.000000   
mean           5270.002907               10.447674       45.494214   
std            1067.959116                2.265895       10.815787   
min            3504.000000                7.000000       32.100000   
25%            4403.000000                9.000000       39.500000   
50%            5106.500000               10.000000       45.100000   
75%            6212.750000               12.000000       49.000000   
max            7197.000000               14.000000      124.300000   

       bill_depth_mm  flipper_length_mm  body_mass_g         year  
count     333.000000         336.000000   339.000000   342.000000  
mean       18.018318         197.764881  4175.463127  2008.035088  
std         9.241384          27.764491   858.713267     0.816938  
min        13.100000          10.000000   882.000000  2007.000000  
25%        15.700000         

In [12]:
penguins_clean = penguins.dropna()
print(penguins_clean.isnull().sum())


species                   0
island                    0
calorie requirement       0
average sleep duration    0
bill_length_mm            0
bill_depth_mm             0
flipper_length_mm         0
body_mass_g               0
gender                    0
year                      0
dtype: int64


In [14]:
numeric_cols = penguins.select_dtypes(include=[np.number]).columns
categorical_cols = penguins.select_dtypes(exclude=[np.number]).columns


In [19]:
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)):
        if not np.isnan(row1[i]) and not np.isnan(row2[i]):
            distance += (row1[i] - row2[i]) ** 2
    return np.sqrt(distance)

def knn_impute_numeric(data, k=5):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if np.isnan(data[i, j]):
                distances = []
                for r in range(data.shape[0]):
                    if r != i and not np.isnan(data[r, j]):
                        distances.append((euclidean_distance(data[i], data[r]), data[r, j]))

                distances.sort(key=lambda x: x[0])
                neighbors = [dist[1] for dist in distances[:k]]
        
                data[i, j] = np.mean(neighbors)
    
    return data
    
penguin_numeric_values = penguins[numeric_cols].values  
penguin_numeric_imputed = knn_impute_numeric(penguin_numeric_values, k=5)

penguins[numeric_cols] = pd.DataFrame(penguin_numeric_imputed, columns=numeric_cols)

for col in categorical_cols:
    penguins[col] = penguins[col].fillna(penguins[col].mode()[0])
print(penguins.isnull().sum())




species                   0
island                    0
calorie requirement       0
average sleep duration    0
bill_length_mm            0
bill_depth_mm             0
flipper_length_mm         0
body_mass_g               0
gender                    0
year                      0
dtype: int64
